In [119]:
import geopandas as gpd
import pandas as pd
import numpy as np
import shapely
from shapely import Polygon, MultiPolygon, LineString, Point
import time

# Read in Data

In [2]:
#takes 45 seconds

gdf = gpd.read_file('data/shapefiles/MAMMALS_TERRESTRIAL_ONLY')

df = pd.read_csv('data/observations.csv')

# Make Mass Table

In [3]:
df_highest_mass = df[df['body mass'].notnull()].sort_values(by = 'body mass', ascending = False)\
                    .drop_duplicates(subset = ['species'], keep = 'first')

masses = gdf[['sci_name']].drop_duplicates().merge(df_highest_mass[['species','body mass']], 
                                                   left_on = 'sci_name', right_on = 'species', how = 'left')\
                                                   .drop(columns = ['species'])

In [4]:
#masses.to_csv('masses_from_ipynb.csv', index = False)

# Read in Masses

In [5]:
masses = pd.read_csv('data/masses_bug.csv')

# Create new Geometries

In [6]:
gdf = gdf.merge(masses, how = 'left', on = 'sci_name').dropna(subset = 'body mass')\
    .sort_values(by = 'body mass', ascending = False)

In [7]:
#import time
#
#df['no_overlap_geometry'] = np.nan
#union = shp([])
#
#start = time.time()
#abs_times = [0]
#rel_times = []
#j = 0
#for i, row in gdf.iterrows():
#    this_geo = row['geometry']
#    new_geo = this_geo - union
#    gdf.loc[i, 'no_overlap_geometry'] = new_geo
#    if new_geo.area > 0:
#        union = this_geo.union(union)
#    j += 1
#    rel_times.append(time.time() - abs_times[-1] - start)
#    abs_times.append(time.time()-start)
#    print(j, rel_times[-1], abs_times[-1])

In [130]:
df['no_overlap_geometry'] = np.nan
union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
j = 0
for i, row in gdf.iterrows():
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    union = this_geo.union(union)
    j += 1
    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    print(j, rel_times[-1], abs_times[-1])

1 0.3199779987335205 0.31998109817504883
2 0.13622403144836426 0.4562070369720459
3 0.21595096588134766 0.6721599102020264
4 0.1595752239227295 0.8317360877990723
5 0.1018528938293457 0.9335899353027344
6 0.22745919227600098 1.1610512733459473
7 0.48723292350769043 1.6482858657836914
8 0.22986125946044922 1.878148078918457
9 0.29496192932128906 2.1731109619140625
10 0.33667898178100586 2.5097920894622803
11 0.2890958786010742 2.7988901138305664
12 0.4225351810455322 3.221426248550415
13 0.3677058219909668 3.5891342163085938
14 0.6466479301452637 4.23578405380249
15 0.7256102561950684 4.961395978927612
16 0.6654760837554932 5.626873016357422
17 0.41362690925598145 6.040501117706299
18 0.6943490505218506 6.734851121902466
19 0.41458797454833984 7.149440050125122
20 0.3618171215057373 7.511258125305176
21 1.3829989433288574 8.894258975982666
22 2.0030300617218018 10.897302150726318
23 1.0250170230865479 11.922320127487183
24 2.007107973098755 13.929430961608887
25 1.5977659225463867 15.52

Stalls on the 386th row, which is Hyaena Hyaena

In [7]:
hyena = gdf['geometry'].iloc[385]

def count_points(geo):
    if isinstance(geo, shapely.geometry.polygon.Polygon):
        res = len(geo.exterior.xy[0])
    else: #must be multipolygon
        res = sum([len(poly.exterior.xy[0]) for poly in geo.geoms])
    return(res)

n_hyaena_points = count_points(hyena)

counts = gdf['geometry'].map(count_points)

print(f"The Striped Hyena Polygon has {n_hyaena_points} points")
print(f"The average polygon has {int(np.mean(counts))} points, \
with standard deviation {int(np.std(counts))}")

The Striped Hyena Polygon has 31637 points
The average polygon has 28558 points, with standard deviation 125323


The Striped Hyena polygon has only an average number of points, that's not the problem

Let's see if this index will work when we do fewer shapes before striped hyenas

In [9]:
df['no_overlap_geometry'] = np.nan
union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
j = 0
for i, row in gdf.iloc[384:400].iterrows():
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    if new_geo.area > 0:
        union = this_geo.union(union)
    j += 1
    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    print(j, row.name, rel_times[-1], abs_times[-1])

1 2456 0.001940011978149414 0.0019409656524658203
2 10904 0.05449819564819336 0.05644106864929199
3 10566 0.10229802131652832 0.15874099731445312
4 108 0.0009548664093017578 0.1596970558166504
5 109 0.03975796699523926 0.19945502281188965
6 5245 0.04575204849243164 0.2452082633972168
7 10519 0.056352853775024414 0.3015623092651367
8 2670 0.08177995681762695 0.3833432197570801
9 2671 0.07647275924682617 0.45981597900390625
10 4998 0.06420302391052246 0.5240199565887451
11 4999 0.06469011306762695 0.5887110233306885
12 4996 0.07158112525939941 0.6602921485900879
13 4995 0.07900404930114746 0.7392971515655518
14 4994 0.1897869110107422 0.9290850162506104
15 5001 0.09305500984191895 1.0221409797668457
16 5000 0.08522391319274902 1.1073648929595947


There is no problem calculating the polygons unless it is with the previous polygon, let's count the number of points

In [11]:
df['no_overlap_geometry'] = np.nan
union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
j = 0
all_pts = 0
for i, row in gdf.iterrows():
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    if new_geo.area > 0:
        union = this_geo.union(union)
    j += 1
    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    these_pts = count_points(this_geo)
    old_pts = all_pts
    all_pts = count_points(union)
    print(j, row.name, row['sci_name'], rel_times[-1], abs_times[-1], these_pts, all_pts - old_pts, all_pts)

1 3782 Loxodonta africana 0.004770994186401367 0.004772186279296875 33 33 33
2 3781 Loxodonta africana 0.03064894676208496 0.03542327880859375 60017 60017 60050
3 3780 Loxodonta africana 0.06588864326477051 0.10131311416625977 10126 3547 63597
4 3779 Loxodonta africana 0.05422496795654297 0.15553808212280273 7706 2420 66017
5 8078 Elephas maximus 0.05407214164733887 0.2096109390258789 114134 114134 180151
6 8077 Elephas maximus 0.17617511749267578 0.3857870101928711 26176 23756 203907
7 8079 Elephas maximus 0.39330101013183594 0.779088020324707 173320 131958 335865
8 11449 Loxodonta cyclotis 0.3133220672607422 1.0924110412597656 34993 34286 370151
9 11450 Loxodonta cyclotis 0.35206103324890137 1.4444739818572998 14518 -1355 368796
10 11451 Loxodonta cyclotis 0.35530710220336914 1.7997832298278809 16783 -3632 365164
11 2714 Ceratotherium simum 0.3195986747741699 2.1193830966949463 32581 28618 393782
12 2715 Ceratotherium simum 0.4691600799560547 2.588545083999634 51461 43695 437477
13 2

AttributeError: 'LineString' object has no attribute 'exterior'

We ran into an error trying to count the points in a linestring! This seems like a piece of the puzzle, because it happens only 10 geometries before our stall.

A linestring is unexpected from these operations, but is technically possible if we tried to intersect two polygons that fit together perfectly along opposite sides of an edge.

Also, a linestring shouldn't cause a problem trying to intersect with a polygon, so it's still not completely clear what's happening. Let's try to boil down the problem to a simpler case to study

In [ ]:
#union.intersect(hyena)

In [ ]:
#union.union(hyena)

Sure enough, if we try to intersect or union this resulting shape with our hyena polygon, we run into the stalling that's causing problems, to make sure we can try doing the operations using the union polygon sans the linestring.

In [15]:
#conveniently, the linestring is the last geometry in the multipolygon
union_noline = MultiPolygon(union.geoms[:-1].geoms)

def has_line(multipoly):
    for geo in multipoly.geoms:
        if isinstance(geo, shapely.geometry.linestring.LineString):
            return(True)
    return(False)

print(has_line(union))
print(has_line(union_noline))

True
False


In [ ]:
union_noline.intersection(hyena)
union_noline.union(hyena)

Running the intersection and union without the linestring works, so we've pretty confidently identified our problem. Let's find the smallest case where we can replicate it.

In [ ]:
unexpected_linestring = union.geoms[-1]
unexpected_linestring.intersection(hyena)
unexpected_linestring.union(hyena)

Just the line alone interacts fine with hyena.

Let's check where in the world our problem is happening

In [43]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [44]:
unexpected_linestring.coords.xy

(array('d', [-71.5502011218265, -71.55020112158445]),
 array('d', [61.17213549303718, 61.17213549305859]))

This is somewhere in India, none of our species have habitats there so how did we get a linestring there?

In [45]:
df['no_overlap_geometry'] = np.nan
union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
all_pts = 0
for i in range(376,0,-1):
    row = gdf.iloc[i]
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    if new_geo.area > 0:
        union = this_geo.union(union)

    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    these_pts = count_points(this_geo)
    old_pts = all_pts
    all_pts = count_points(union)
    print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1], these_pts, all_pts - old_pts, all_pts)

377 292 Canis lupus 0.0153961181640625 0.015398979187011719 206245 206245 206245
376 293 Canis lupus 0.21044301986694336 0.2258439064025879 2618 2618 208863
375 294 Canis lupus 0.15652203559875488 0.3823668956756592 687 687 209550
374 295 Canis lupus 0.15260815620422363 0.5349769592285156 1923 1923 211473
373 296 Canis lupus 0.1657700538635254 0.7007491588592529 8184 8184 219657
372 297 Canis lupus 0.1560969352722168 0.8568470478057861 622 622 220279
371 284 Canis lupus 0.16025090217590332 1.0170989036560059 3615 3615 223894
370 283 Canis lupus 0.15372014045715332 1.1708199977874756 45 45 223939
369 282 Canis lupus 0.16641998291015625 1.3372421264648438 1852 1852 225791
368 281 Canis lupus 0.1701047420501709 1.5073480606079102 3311 3311 229102
367 280 Canis lupus 0.1457059383392334 1.65305495262146 35 6 229108
366 475 Vombatus ursinus 0.0415952205657959 1.6946511268615723 4075 4075 233183
365 476 Vombatus ursinus 0.04089784622192383 1.7355499267578125 3665 3665 236848
364 477 Vombatus 

AttributeError: 'LineString' object has no attribute 'exterior'

Intersecting rows between 344 and 377 gives us a polygon with a linestring, let's see if the linestring is in the same location and if this union is equally problematic

In [47]:
another_unexpected_linestring = union.geoms[-1]
another_unexpected_linestring.coords.xy

(array('d', [15.078850600774672, 15.078850599875295]),
 array('d', [-26.671422199774668, -26.67142219977461]))

In [ ]:
union.union(hyena)
union.intersection(hyena)

1) This linestring is in a seperate location, this time in Sudan, about 150 miles northwest from Al-Fashir

2) This union actually works, so we are still missing the piece in conjunction with the linestring that causes the bug, let's build up this polygon until it can't intersect or union with hyena

In [66]:
#df['no_overlap_geometry'] = np.nan
#union = Polygon([])

#start = time.time()
#abs_times = [0]
#rel_times = []
#all_pts = 0
#for i in range(376,0,-1):
#    row = gdf.iloc[i]
#    this_geo = row['geometry']
#    new_geo = this_geo - union
#    gdf.loc[i, 'no_overlap_geometry'] = new_geo
#    if new_geo.area > 0:
#        union = this_geo.union(union)

#    rel_times.append(time.time() - abs_times[-1] - start)
#    abs_times.append(time.time()-start)
#    
#    print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1])
#
#    union.union(hyena)
#    union.intersection(hyena)

377 292 Canis lupus 0.006639957427978516 0.0066411495208740234
376 293 Canis lupus 0.35474157333374023 0.36138486862182617
375 294 Canis lupus 0.3392329216003418 0.7006189823150635
374 295 Canis lupus 0.32563304901123047 1.0262529850006104
373 296 Canis lupus 0.35061097145080566 1.3768658638000488
372 297 Canis lupus 0.3508272171020508 1.727694034576416
371 284 Canis lupus 0.3616938591003418 2.0893890857696533
370 283 Canis lupus 0.34471797943115234 2.4341087341308594
369 282 Canis lupus 0.36125922203063965 2.7953689098358154
368 281 Canis lupus 0.356950044631958 3.15231990814209
367 280 Canis lupus 0.33643603324890137 3.4887568950653076
366 475 Vombatus ursinus 0.2323169708251953 3.7210748195648193
365 476 Vombatus ursinus 0.23903822898864746 3.960113048553467
364 477 Vombatus ursinus 0.3620278835296631 4.322141885757446
363 2930 Acinonyx jubatus 0.8481509685516357 5.1702940464019775
362 2931 Acinonyx jubatus 1.0414979457855225 6.211793899536133
361 2932 Acinonyx jubatus 1.11634397506

Combinining rows 377 to  337 gives us the pathological shape

In [128]:
df['no_overlap_geometry'] = np.nan
union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
all_pts = 0
for i in range(376,335,-1):
    row = gdf.iloc[i]
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    if new_geo.area > 0:
        union = this_geo.union(union)
    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    
    print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1])

#    union.union(hyena)
#    union.intersection(hyena)

377 292 Canis lupus 0.018353939056396484 0.0183560848236084
376 293 Canis lupus 0.12893104553222656 0.14728999137878418
375 294 Canis lupus 0.23594403266906738 0.38323521614074707
374 295 Canis lupus 0.1338357925415039 0.5170719623565674
373 296 Canis lupus 0.14313316345214844 0.6602070331573486
372 297 Canis lupus 0.1311650276184082 0.7913732528686523
371 284 Canis lupus 0.13269972801208496 0.9240741729736328
370 283 Canis lupus 0.11683201789855957 1.0409071445465088
369 282 Canis lupus 0.13362693786621094 1.1745359897613525
368 281 Canis lupus 0.13774418830871582 1.312281847000122
367 280 Canis lupus 0.11166906356811523 1.4239518642425537
366 475 Vombatus ursinus 0.004290103912353516 1.4282419681549072
365 476 Vombatus ursinus 0.002885103225708008 1.4311270713806152
364 477 Vombatus ursinus 0.12380695343017578 1.5549352169036865
363 2930 Acinonyx jubatus 0.6726677417755127 2.2276041507720947
362 2931 Acinonyx jubatus 0.5352387428283691 2.7628440856933594
361 2932 Acinonyx jubatus 0.4

In [ ]:
union.intersection(hyena)
union.union(hyena)

Our minimum case right now involves just the territories of the canis lupus (gray wolf), vombatus ursinus (common wombat), acinonyx jubatus (cheetah), myrmecophaga tridactyla (giant anteater), and panthera pardus(leopard), interacting with the territory of hyaena hyaena (striped hyena)

In [66]:
has_line(union)

True

In [48]:
#again conveniently, the linestring is the last geometry in the multipolygon
union_noline = MultiPolygon(union.geoms[:-1].geoms)

print(has_line(union))
print(has_line(union_noline))

True
False


In [ ]:
union_noline.intersection(hyena)
union_noline.union(hyena)

Again removing the line removes all problems

The species involved that have territory in Sudan where we are making the linestring are the cheetah and the leopard, with the leopard boundary looking like it aligns with the problematic linestring. Let's see if we can shrink our minimal case by using just those two species

In [67]:
species = ['Acinonyx jubatus','Panthera pardus']

union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []
all_pts = 0
for i in range(376,335,-1):
    row = gdf.iloc[i]
    if row['sci_name'] in species:
        this_geo = row['geometry']
        new_geo = this_geo - union
        if new_geo.area > 0:
            union = this_geo.union(union)
        rel_times.append(time.time() - abs_times[-1] - start)
        abs_times.append(time.time()-start)
    
        print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1])

#    union.union(hyena)
#    union.intersection(hyena)

363 2930 Acinonyx jubatus 0.009398937225341797 0.00940084457397461
362 2931 Acinonyx jubatus 0.4371812343597412 0.4465827941894531
361 2932 Acinonyx jubatus 0.3013792037963867 0.7479629516601562
360 2934 Acinonyx jubatus 2.9693429470062256 3.7173070907592773
356 2929 Acinonyx jubatus 2.4073760509490967 6.124690055847168
355 2933 Acinonyx jubatus 3.3966619968414307 9.521360874176025
354 6213 Panthera pardus 0.017963171005249023 9.539325952529907
353 6212 Panthera pardus 2.7259809970855713 12.26530909538269
352 6211 Panthera pardus 2.601510763168335 14.866822957992554
351 6214 Panthera pardus 0.039459228515625 14.906283140182495
350 6203 Panthera pardus 2.625882863998413 17.532169103622437
349 6204 Panthera pardus 2.6515750885009766 20.183745861053467
348 6218 Panthera pardus 2.605452060699463 22.78920006752014
347 6217 Panthera pardus 2.581083059310913 25.37028408050537
346 6210 Panthera pardus 3.179795980453491 28.550083875656128
345 6209 Panthera pardus 3.0961201190948486 31.646210908

In [68]:
has_line(union)

False

This union alone does not have the line, so it probably won't cause a problem right?

In [ ]:
union.intersection(hyena)
union.union(hyena)

No problem yet! Let's add in more creatures until we get a line.

In [125]:
species = ['Acinonyx jubatus','Panthera pardus','Canis lupus']

union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []

for i in range(376,335,-1):
    row = gdf.iloc[i]
    if row['sci_name'] in species:
        this_geo = row['geometry']
        union = this_geo.union(union)
        rel_times.append(time.time() - abs_times[-1] - start)
        abs_times.append(time.time()-start)
        
        print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1], has_line(union))

#    union.union(hyena)
#    union.intersection(hyena)

377 292 Canis lupus 0.0039501190185546875 0.0039539337158203125 False
376 293 Canis lupus 0.13151812553405762 0.13547396659851074 False
375 294 Canis lupus 0.11233019828796387 0.24780488014221191 False
374 295 Canis lupus 0.11202812194824219 0.359835147857666 False
373 296 Canis lupus 0.11368083953857422 0.47351694107055664 False
372 297 Canis lupus 0.11559009552001953 0.5891079902648926 False
371 284 Canis lupus 0.11650300025939941 0.705610990524292 False
370 283 Canis lupus 0.1194150447845459 0.8250269889831543 False
369 282 Canis lupus 0.1286149024963379 0.9536440372467041 False
368 281 Canis lupus 0.12017107009887695 1.0738170146942139 False
367 280 Canis lupus 0.10943484306335449 1.1832530498504639 False
363 2930 Acinonyx jubatus 0.3812389373779297 1.56449294090271 False
362 2931 Acinonyx jubatus 0.3607912063598633 1.9252851009368896 False
361 2932 Acinonyx jubatus 0.36493802070617676 2.2902252674102783 False
360 2934 Acinonyx jubatus 2.9350945949554443 5.225329875946045 False
356

In [126]:
has_line(union)

True

In [127]:
unexpected_linestring = union.geoms[0]
unexpected_linestring.coords.xy

(array('d', [15.078850600774672, 15.078850599875295]),
 array('d', [-26.671422199774668, -26.67142219977461]))

In order to get a linestring, we need all three of cheetah, leopard, and wolf. But how does the wolf territory, which doesn't intersect Sudan, cause a linestring in Sudan?

In [ ]:
union.union(hyena)
union.intersection(hyena)

In [ ]:
species = ['Acinonyx jubatus','Panthera pardus','Canis lupus']

union = Polygon([])

start = time.time()
abs_times = [0]
rel_times = []

for i in range(336,377):
    row = gdf.iloc[i]
    if row['sci_name'] in species:
        this_geo = row['geometry']
        new_geo = this_geo - union
        if new_geo.area > 0:
            union = this_geo.union(union)
        rel_times.append(time.time() - abs_times[-1] - start)
        abs_times.append(time.time()-start)
        
        print(i+1, row.name, row['sci_name'], rel_times[-1], abs_times[-1])

#    union.union(hyena)
#    union.intersection(hyena)

In [ ]:
union = Polygon([])
for geom in gdf[gdf['sci_name'] == 'Canis lupus']['geometry']:
    if isinstance(geom, shapely.geometry.polygon.Polygon):
        print(1)
    else:
        print(len(geom.geoms))
    union = union.union(geom)
union

How does the wolf territory, which does not seem to touch Sudan, 

In [ ]:
union

In [81]:
unexpected_linestring = union.geoms[-1]
unexpected_linestring.coords.xy

NotImplementedError: Component rings have coordinate sequences, but the polygon does not

In [ ]:
#test case for intersecting a linestring and a polygon, generated by ChatGPT

from shapely.geometry import LineString, Polygon

def test_intersect_line_polygon():
    # Test data
    line = LineString([(0, 0), (5, 5)])
    polygon = Polygon([(2, 2), (2, 8), (8, 8), (8, 2)])

    # Execute
    intersection = line.intersection(polygon)

    # Assertion
    assert intersection.is_empty == False, "Intersection should not be empty"
    assert intersection.geom_type == 'LineString' or intersection.geom_type == 'MultiLineString', "Intersection should be a LineString or MultiLineString"

    return(intersection)

# Run the test
test_intersect_line_polygon()

There is no sharp spike in points before Hyena either. It must be something more subtle about the polygon. Let's look at the geometries

In [ ]:
len(gdf.iloc[376]['geometry'].geoms)

7

In [ ]:
for i in gdf.iloc[376]['geometry'].geoms:
    print(type(i))

<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>


In [ ]:
union.geoms[-1].coords.xy

NotImplementedError: Component rings have coordinate sequences, but the polygon does not

In [ ]:
gdf.iloc[376]['geometry']

In [ ]:
for i in union.geoms:
    print(type(i))

<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'shapely.geometry.polygon.Polygon'>
<class 'sha

In [ ]:
gdf.iloc[2]['geometry'].geoms[0]

In [ ]:
gdf.iloc[370]['geometry']

Everything slowed down after index 345

In [ ]:
gdf.iloc[344:345]

,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,subpop,...,category,marine,terrestial,freshwater,Shape_Leng,Shape_Area,geometry,body mass,no_overlap_geometry,n_points
6209,15954,Panthera pardus,5,1,1,Peter Gerngross,2023,"Bleyhl, P., Gerngross, P., et al.",fusca,None,...,VU,false,true,false,512.486984,229.289417,"MULTIPOLYGON (((74.70788 13.69233, 74.71104 13...",54.0,MULTIPOLYGON (((74.71104319998375 13.691558900...,127304


In [ ]:
gdf.iloc[340:400]

,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,subpop,...,genus,category,marine,terrestial,freshwater,Shape_Leng,Shape_Area,geometry,body mass,no_overlap_geometry
6201,15954,Panthera pardus,5,1,1,Bleyhl & Gerngross et al.,2023,"Bleyhl, P., Gerngross, P., et al.",tulliana,None,...,Panthera,VU,false,true,false,650.782073,230.689269,"MULTIPOLYGON (((46.09309 34.05112, 46.12457 34...",54.0,MULTIPOLYGON (((46.124567225197325 34.03252162...
6202,15954,Panthera pardus,1,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",melas,None,...,Panthera,VU,false,true,false,44.491206,0.837157,"MULTIPOLYGON (((114.39599 -8.48946, 114.39613 ...",54.0,MULTIPOLYGON (((114.39612788519378 -8.49003949...
6207,15954,Panthera pardus,5,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",kotiya,None,...,Panthera,VU,false,true,false,20.758407,2.069591,"MULTIPOLYGON (((81.21515 6.69759, 81.09985 6.6...",54.0,MULTIPOLYGON (((81.09985286991406 6.6743944579...
6215,15954,Panthera pardus,1,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",pardus,None,...,Panthera,VU,false,true,false,773.433355,553.918782,"MULTIPOLYGON (((21.12695 -33.43875, 21.15281 -...",54.0,MULTIPOLYGON (((27.27025604248246 -33.41716003...
6209,15954,Panthera pardus,5,1,1,Peter Gerngross,2023,"Bleyhl, P., Gerngross, P., et al.",fusca,None,...,Panthera,VU,false,true,false,512.486984,229.289417,"MULTIPOLYGON (((74.70788 13.69233, 74.71104 13...",54.0,MULTIPOLYGON (((74.71104319998375 13.691558900...
6210,15954,Panthera pardus,3,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",pardus,None,...,Panthera,VU,false,true,false,68.916335,14.081474,"MULTIPOLYGON (((36.55102 -15.78752, 36.55026 -...",54.0,MULTIPOLYGON (((37.42839199989095 18.859234399...
6217,15954,Panthera pardus,3,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",None,None,...,Panthera,VU,false,true,false,25.379054,5.122939,"MULTIPOLYGON (((115.97439 24.80619, 115.96517 ...",54.0,MULTIPOLYGON (((115.96516883297534 24.80607294...
6218,15954,Panthera pardus,1,1,1,Hadi Al Hikmani,2023,"Bleyhl, P., Gerngross, P., et al.",nimr,None,...,Panthera,VU,false,true,false,9.018944,0.951815,"MULTIPOLYGON (((45.58200 13.92015, 45.69156 13...",54.0,MULTIPOLYGON (((45.691555926764636 13.90712646...
6204,15954,Panthera pardus,3,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",melas,None,...,Panthera,VU,false,true,false,11.217410,0.150362,"MULTIPOLYGON (((114.21742 -8.16347, 114.21742 ...",54.0,MULTIPOLYGON (((114.21741567659228 -8.17689676...
6203,15954,Panthera pardus,5,1,1,Wibisono et al. 2018,2023,"Bleyhl, P., Gerngross, P., et al.",melas,None,...,Panthera,VU,false,true,false,79.002355,9.336906,"MULTIPOLYGON (((114.54298 -8.78026, 114.53639 ...",54.0,MULTIPOLYGON (((114.53639047364152 -8.78039208...


In [ ]:
type(gdf['geometry'].values[12])

shapely.geometry.polygon.Polygon

In [ ]:
gdf['geometry'].values[12] is shapely.geometry.polygon.Polygon

False

In [ ]:
isinstance(gdf['geometry'].values[12], shapely.geometry.polygon.Polygon)

True

In [ ]:
gdf['n_pts'] = gdf['geometry'].map(count_points)

In [ ]:
gdf_df = pd.DataFrame(gdf[['sci_name','body mass','n_pts']])

In [ ]:
geo1 = geo.geoms[0]

In [ ]:
len(geo1.exterior.xy[0])

5

In [ ]:
geo1.exterior

AttributeError: 'Polygon' object has no attribute 'interior'

In [ ]:
type(geo.geoms[4])

shapely.geometry.polygon.Polygon

In [ ]:
gdf

,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,subpop,...,family,genus,category,marine,terrestial,freshwater,Shape_Leng,Shape_Area,geometry,body mass
3782,181008073,Loxodonta africana,1,2,1,IUCN SSC African Elephant Specialist Group,2021,IUCN SSC African Elephant Specialist Group,None,None,...,ELEPHANTIDAE,Loxodonta,EN,false,true,false,0.310021,0.002028,"MULTIPOLYGON (((31.76359 -26.61175, 31.76487 -...",6000.000
3781,181008073,Loxodonta africana,1,1,1,IUCN SSC African Elephant Specialist Group,2021,IUCN SSC African Elephant Specialist Group,None,None,...,ELEPHANTIDAE,Loxodonta,EN,false,true,false,518.449852,139.321987,"MULTIPOLYGON (((22.97502 -34.07926, 22.97567 -...",6000.000
3780,181008073,Loxodonta africana,4,1,1,IUCN SSC African Elephant Specialist Group,2021,IUCN SSC African Elephant Specialist Group,None,None,...,ELEPHANTIDAE,Loxodonta,EN,false,true,false,271.889957,69.553075,"MULTIPOLYGON (((29.23323 -21.74140, 29.26674 -...",6000.000
3779,181008073,Loxodonta africana,3,1,1,IUCN SSC African Elephant Specialist Group,2021,IUCN SSC African Elephant Specialist Group,None,None,...,ELEPHANTIDAE,Loxodonta,EN,false,true,false,223.518362,58.928504,"MULTIPOLYGON (((32.47534 -20.13611, 32.48143 -...",6000.000
8078,7140,Elephas maximus,3,5,1,"IUCN/SSC AsESG, WCS, WWF",2020,"IUCN/SSC AsESG, WCS, WWF",None,None,...,ELEPHANTIDAE,Elephas,EN,false,true,false,246.534093,9.490031,"MULTIPOLYGON (((103.81783 -4.38439, 103.77385 ...",4500.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,29662,Sorex caecutiens,1,1,1,IUCN,2012,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,0.950475,0.050470,"POLYGON ((156.40591 50.64036, 156.40639 50.638...",0.010
965,41422,Sorex tundrensis,1,1,1,IUCN,2012,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,953.025010,2698.008609,"MULTIPOLYGON (((81.54186 45.11874, 81.72656 45...",0.010
2355,29667,Sorex minutus,1,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,1357.221893,2064.400425,"MULTIPOLYGON (((83.64068 26.61991, 83.28411 26...",0.004
2354,29667,Sorex minutus,2,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,7.927799,1.513192,"POLYGON ((75.49854 32.28351, 75.37968 32.25822...",0.004


In [ ]:
un = shp([(0,0),(0,1),(1,1),(1,0)]).union(shp([(0.1,0.1),(0.1,0.9),(0.9,0.9),(0.9,0.1)]))

In [ ]:
un.exterior.coords.xy

(array('d', [0.0, 1.0, 1.0, 0.0, 0.0]), array('d', [1.0, 1.0, 0.0, 0.0, 1.0]))

In [ ]:
geo = gdf['geometry'].values[385]
geo

In [ ]:
type(gdf.index[385])

numpy.int64

In [ ]:
len(gdf)

568

In [ ]:
568 - 386

182

In [ ]:
gdf[gdf.index == 10904]

,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,subpop,...,family,genus,category,marine,terrestial,freshwater,Shape_Leng,Shape_Area,geometry,body mass
10904,10274,Hyaena hyaena,1,1,1,IUCN,2015,IUCN (International Union for Conservation of ...,None,None,...,HYAENIDAE,Hyaena,NT,false,true,false,618.081256,2084.148213,"MULTIPOLYGON (((39.16920 21.15934, 39.17740 21...",35.0


In [ ]:
gdf.iloc[385]

id_no                                                     10274
sci_name                                          Hyaena hyaena
presence                                                      1
origin                                                        1
seasonal                                                      1
compiler                                                   IUCN
yrcompiled                                                 2015
citation      IUCN (International Union for Conservation of ...
subspecies                                                 None
subpop                                                     None
source                                                     None
island                                                     None
tax_comm                                                   None
dist_comm                                                  None
generalisd                                                    0
legend                                  

In [ ]:
gdf.tail(300)

,id_no,sci_name,presence,origin,seasonal,compiler,yrcompiled,citation,subspecies,subpop,...,family,genus,category,marine,terrestial,freshwater,Shape_Leng,Shape_Area,geometry,body mass
4242,30208,Damaliscus pygargus,1,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,phillipsi,None,...,BOVIDAE,Damaliscus,LC,false,true,false,26.122688,35.124665,"POLYGON ((29.04400 -24.40286, 29.09550 -24.420...",100.000
4241,30208,Damaliscus pygargus,1,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,BOVIDAE,Damaliscus,LC,false,true,false,9.129780,1.378003,"MULTIPOLYGON (((18.91650 -34.34826, 18.96740 -...",100.000
140,42394,Odocoileus virginianus,1,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,CERVIDAE,Odocoileus,LC,false,true,false,962.459398,1493.974401,"MULTIPOLYGON (((-74.34519 -4.74012, -74.22961 ...",100.000
4965,15953,Panthera onca,1,1,1,Panthera,2017,Panthera,None,None,...,FELIDAE,Panthera,NT,false,true,false,42.258161,6.638939,"MULTIPOLYGON (((-53.71776 -26.56016, -53.71096...",85.000
4963,15953,Panthera onca,1,1,1,Panthera,2017,Panthera,None,None,...,FELIDAE,Panthera,NT,false,true,false,133.396872,71.281650,"MULTIPOLYGON (((-76.95203 8.22699, -76.93861 8...",85.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3563,29662,Sorex caecutiens,1,1,1,IUCN,2012,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,0.950475,0.050470,"POLYGON ((156.40591 50.64036, 156.40639 50.638...",0.010
965,41422,Sorex tundrensis,1,1,1,IUCN,2012,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,953.025010,2698.008609,"MULTIPOLYGON (((81.54186 45.11874, 81.72656 45...",0.010
2355,29667,Sorex minutus,1,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,1357.221893,2064.400425,"MULTIPOLYGON (((83.64068 26.61991, 83.28411 26...",0.004
2354,29667,Sorex minutus,2,1,1,IUCN,2008,IUCN (International Union for Conservation of ...,None,None,...,SORICIDAE,Sorex,LC,false,true,false,7.927799,1.513192,"POLYGON ((75.49854 32.28351, 75.37968 32.25822...",0.004


In [ ]:
import time

df['no_overlap_geometry'] = np.nan
union = shp([])

start = time.time()
abs_times = [0]
rel_times = []
j = 0
for i, row in gdf.tail(300).iterrows():
    this_geo = row['geometry']
    new_geo = this_geo - union
    gdf.loc[i, 'no_overlap_geometry'] = new_geo
    if new_geo.area > 0:
        union = this_geo.union(union)
    j += 1
    rel_times.append(time.time() - abs_times[-1] - start)
    abs_times.append(time.time()-start)
    print(j, rel_times[-1], abs_times[-1])

1 0.007014274597167969 0.00701594352722168
2 0.031209945678710938 0.03822803497314453
3 0.042099952697753906 0.08032894134521484
4 0.03934526443481445 0.1196751594543457
5 0.03868913650512695 0.15836429595947266
6 0.12556695938110352 0.2839322090148926
7 0.3390798568725586 0.6230132579803467
8 0.20642304420471191 0.8294363021850586
9 0.2448277473449707 1.0742661952972412
10 0.2839019298553467 1.3581690788269043
11 0.24001479148864746 1.5981850624084473
12 0.381544828414917 1.9797320365905762
13 0.3430449962615967 2.3227791786193848
14 0.6022007465362549 2.9249820709228516
15 0.6867351531982422 3.6117188930511475
16 0.6277351379394531 4.239454984664917
17 0.39327311515808105 4.6327290534973145
18 0.6516389846801758 5.284369945526123
19 0.38251614570617676 5.666887998580933
20 0.023475170135498047 5.690364122390747
21 1.29606294631958 6.986428260803223
22 1.5304210186004639 8.516865015029907
23 0.020978927612304688 8.537845134735107
24 1.7651140689849854 10.302961111068726
25 1.217183113

In [ ]:
def half_polygon(poly)
#remove every other point
    xy = poly.exterior.xy
    ne